In [6]:
!pip install -U transformers


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
from transformers import (
    AutoTokenizer,
)
import os

tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit")

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [5]:
def mapper(x):
    clean_chat = []
    x = eval(x)
    for ll in x:
        if len(ll.keys()) != 2:
            clean_chat.append({
                'role': ll['role'],
                'content': ll['content'],
            })
        else:
            clean_chat.append(ll)
    return clean_chat

In [7]:
for dataset in ['DABL01_VTrain.csv', 'DABL01_VTest.csv']:
    path = os.path.join('../../data/training_data', dataset)
    df = pd.read_csv(path)
    df['chat'] = df.chat.map(lambda x: mapper(x))
    df["text"] = df.chat.map(lambda x: tokenizer.apply_chat_template(conversation=x, tokenize=False))
    new_path = os.path.join('../../data/training_data', dataset.split('.')[0] + '_mistral-small.csv')
    df.to_csv(path, index=False)

In [8]:
df.head()

,Unnamed: 0,game,benchmark_version,game_id,model,experiment,episode,Aborted,Lose,Success,chat,text
0,0,imagegame,v1.6,19,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_19,0,0,1,"[{'role': 'user', 'content': 'Let us draw some...",<|begin_of_text|><|start_header_id|>system<|en...
1,1,imagegame,v1.6,19,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_19,0,0,1,"[{'role': 'user', 'content': 'Let us draw some...",<|begin_of_text|><|start_header_id|>system<|en...
2,2,imagegame,v1.6,19,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_19,0,0,1,"[{'role': 'user', 'content': 'Let us draw some...",<|begin_of_text|><|start_header_id|>system<|en...
3,3,imagegame,v1.6,19,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_19,0,0,1,"[{'role': 'user', 'content': 'Let us draw some...",<|begin_of_text|><|start_header_id|>system<|en...
4,4,referencegame,v1.6,21,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,3_letter_grids,episode_21,0,0,1,"[{'role': 'user', 'content': 'You are given th...",<|begin_of_text|><|start_header_id|>system<|en...


In [10]:
df.text.iloc[0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nLet us draw something together. There is an empty grid with a size 5 by 5, like so:\n\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n\nI will give you instructions like "put an X in the top left", and you return the grid by applying the given instruction, like so:\n\nInstruction: put an X in the top left\n\nX ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n\nNow create an empty grid with a size 5 by 5 and execute the following commands at each step. Once you execute the command, return only the grid and exclude all other text from the output.\nInstruction: Put an H in the second row, third column<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n▢ ▢ ▢ ▢ ▢\n▢ ▢ H ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢<|eot_id|>'

In [3]:
import pandas as pd
import os
from datasets import Dataset, load_dataset
df = pd.read_csv(os.path.join('../../data/training_data', 'DFINAL_qwen-coder.csv'))
ds: Dataset = Dataset.from_pandas(df=df)

In [36]:
for i,c in enumerate(list(df.chat)):
    if not isinstance(c, str):
        print(c, i)

In [55]:
for d in ds["chat"]:
    if not isinstance(d, str):
        print(d)

In [7]:
print(df.text.iloc[0])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Let us draw something together. There is an empty grid with a size 5 by 5, like so:

▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢

I will give you instructions like "put an X in the top left", and you return the grid by applying the given instruction, like so:

Instruction: put an X in the top left

X ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢

Or for another instruction such as "fill the fifth column with T", you return the updated grid by applying the given instruction in all places that the command corresponds to, like so:

Instruction: fill the fifth column with T

X ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T

Or for another instruction such as "fill the fourth column second row with P", you return the updated grid by applying the given instruction in all places that the command corresponds to, like so:

Instruction: fill the fourth column second row with P

X ▢ ▢ ▢ T
▢ ▢ ▢ P T
▢ ▢ ▢ ▢ T
▢ ▢ ▢